In [ ]:
import numpy as np

from cc_catheter import CCCatheter

In [ ]:
p_0 = np.array([2e-2, 2e-3, 0])
r = 0.01
n_iter = 20
n_data = 1 # 100
noise_percentage = 0.25
ux_init = 0.00001
uy_init = 0.00001
l = 0.2

'dof': 3, 
loss_2d = False
'tip_loss': True,
'use_reconstruction': True,
'interspace': 1,
'viewpoint_mode': 1,
'damping_weights': [1, 1, 10],
'n_mid_points': 1

In [ ]:
# ----- Create a catheter class, initialize control parameters and target, camera parameters -----
        
catheter = CCCatheter(p_0, l, r, self.loss_2d, self.tip_loss, self.n_mid_points, self.n_iter, verbose=0)
catheter.set_weight_matrix(self.damping_weights[0], self.damping_weights[1], self.damping_weights[2])

if self.use_2d_pos_target:
    assert self.ux
    assert self.uy
    assert self.x_target
    assert self.y_target

    if self.dof == 2:
        catheter.set_2dof_params(self.ux, self.uy)
    elif self.dof == 3:
        catheter.set_3dof_params(self.ux, self.uy, self.l)
    else:
        print('[ERROR] DOF invalid')

    catheter.set_2d_targets([0, self.x_target], [0, self.y_target])

else:
    if self.dof == 1:
        assert self.u
        assert self.phi
        assert self.u_target

        catheter.set_1dof_params(self.phi, self.u)
        catheter.set_1dof_targets(self.u_target)

    elif self.dof == 2:
        assert self.ux
        assert self.uy
        assert self.ux_target
        assert self.uy_target

        catheter.set_2dof_params(self.ux, self.uy)
        catheter.set_2dof_targets(self.ux_target, self.uy_target)

    elif self.dof == 3:
        assert self.ux
        assert self.uy
        assert self.ux_target
        assert self.uy_target
        assert self.l_target

        catheter.set_3dof_params(self.ux, self.uy, self.l)
        catheter.set_3dof_targets(self.ux_target, self.uy_target, self.l_target)

    else:
        print('[ERROR] DOF invalid')

catheter.set_camera_params(camera_settings.a, camera_settings.b, camera_settings.center_x, camera_settings.center_y, camera_settings.image_size_x, camera_settings.image_size_y, camera_settings.extrinsics)

# ----- Apply motion model to simulate real catheter behavior (initial step) -----
catheter.calculate_cc_points(init=True, noise_percentage=0.05)
catheter.convert_cc_points_to_2d(init=True)
catheter.calculate_beziers_control_points(noise_level=0.007)

if not self.use_2d_pos_target: # if use 3D target
    catheter.calculate_cc_points(target=True)
    catheter.convert_cc_points_to_2d(target=True)

cc_specs_path = os.path.join(self.cc_specs_save_dir , '000.npy')
image_save_path = os.path.join(self.images_save_dir, '000.png')

if self.use_reconstruction or self.use_2d_pos_target:
    bezier_specs_old = catheter.calculate_bezier_specs(init=True)
    target_specs_path = None
else:
    target_specs_path = os.path.join(self.cc_specs_save_dir, 'target.npy')
    catheter.write_target_specs(target_specs_path, show_mid_points=True)